## Apache Iceberg

**Apache Iceberg is an open table format for huge analytic datasets**. Iceberg adds tables to compute engines including Spark, Trino, PrestoDB, Flink, Hive and Impala using a high-performance table format that works just like a SQL table.

- Iceberg is designed for huge tables and is used in production where a single table can contain tens of petabytes of data.
- Even multi-petabyte tables can be read from a single node, without needing a distributed SQL engine to sift through table metadata.


In [1]:
from pyspark.sql import SparkSession

catalog_type = "hadoop"  # directory-based catalog in HDFS
warehouse_path = "/iceberg-warehouse"  # path to warehouse in HDFS hdfs://localhost:9000/iceberg-warehouse

# Inicializar a SparkSession com suporte ao Iceberg
spark = SparkSession.builder \
    .appName("Iceberg - HDFS") \
    .config("spark.executor.memory", "1G") \
    .config("spark.driver.memory", "1G") \
    .config("spark.driver.maxResultSize", "1G") \
    .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0") \
    .config("spark.sql.catalog.iceberg_catalog", "org.apache.iceberg.spark.SparkCatalog") \
    .config("spark.sql.catalog.iceberg_catalog.type", catalog_type) \
    .config("spark.sql.catalog.iceberg_catalog.warehouse", warehouse_path) \
    .config("spark.sql.defaultCatalog", "iceberg_catalog") \
    .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")
spark


:: loading settings :: url = jar:file:/opt/spark-3.5.1-bin-without-hadoop/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-835df365-22f4-4660-89dc-70cf280c6c4e;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.0 in central
:: resolution report :: resolve 56ms :: artifacts dl 1ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submi

In [2]:
spark.sql("show catalogs").toPandas()

,catalog
0,iceberg_catalog
1,spark_catalog


In [3]:
spark.catalog.listCatalogs()

[CatalogMetadata(name='iceberg_catalog', description=None),
 CatalogMetadata(name='spark_catalog', description=None)]

In [4]:
spark.catalog.currentCatalog()

'iceberg_catalog'

In [5]:
# create database/namespace
spark.sql("CREATE NAMESPACE IF NOT EXISTS mydb").toPandas()

""


In [6]:
spark.sql("show databases").toPandas()

,namespace
0,mydb


In [7]:
spark.catalog.listDatabases()

[Database(name='mydb', catalog='iceberg_catalog', description=None, locationUri='/iceberg-warehouse/mydb')]

In [8]:
spark.sql("describe database mydb").toPandas()

,info_name,info_value
0,Catalog Name,iceberg_catalog
1,Namespace Name,mydb
2,Location,/iceberg-warehouse/mydb


### Create a Table

In [9]:
spark.sql(f"""
CREATE OR REPLACE TABLE mydb.users (
    id INT COMMENT 'Identificador único',
    name STRING COMMENT 'Nome do indivíduo',
    updated_at date COMMENT 'Data de update'
)
USING iceberg
""")

DataFrame[]

In [10]:
spark.sql("show tables in mydb").toPandas()

,namespace,tableName,isTemporary
0,mydb,users,False


In [11]:
spark.sql("""
    describe table extended mydb.users
""").toPandas()

,col_name,data_type,comment
0,id,int,Identificador único
1,name,string,Nome do indivíduo
2,updated_at,date,Data de update
3,,,
4,# Metadata Columns,,
5,_spec_id,int,
6,_partition,struct<>,
7,_file,string,
8,_pos,bigint,
9,_deleted,boolean,


In [13]:
from datetime import datetime

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType
from pyspark.sql.functions import lit, col
from delta.tables import DeltaTable

In [14]:
data =  [{'id': 1, 'name': 'Alice', 'updated_at': datetime(2022, 1, 1)},
         {'id': 2, 'name': 'Braga', 'updated_at': datetime(2022, 2, 2)},
         {'id': 3, 'name': 'Steve', 'updated_at': datetime(2022, 3, 3)}]

schema = StructType([StructField('id', IntegerType(), nullable=True),
                     StructField('name', StringType(), nullable=True),
                     StructField('updated_at', DateType(), nullable=True)])

df = spark.createDataFrame(data, schema=schema)
df.toPandas()

,id,name,updated_at
0,1,Alice,2022-01-01
1,2,Braga,2022-02-02
2,3,Steve,2022-03-03


In [15]:
df.writeTo("mydb.users").append()

In [16]:
spark.table("mydb.users").toPandas()

,id,name,updated_at
0,1,Alice,2022-01-01
1,2,Braga,2022-02-02
2,3,Steve,2022-03-03
